In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import *
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold


In [3]:
cd "C:\Users\Theo\Downloads"

C:\Users\Theo\Downloads


In [16]:
def create_table(name_of_file):
    dictionary=dict()
    with open(name_of_file,"r") as data:
        for number,line in enumerate(data):
            instructions=[]
            instr= line.strip('{"instructions": [').split("],")[0].replace('"','')
            for i in range(len(instr.split(', '))):
                try:
                    instructions.append(instr.split(', ')[i].split()[0])

                except:
                    print('error or instance  '+str(i)+ 'and number  '+str(number)+ 'with text:   '+ str(instr.split(', ')[i].split()))

            rest= line.strip('{"instructions": [').split("],")[1].replace('"','').replace('}','').replace(',','').split()
                #print(rest[1]) # number 1 is the optimizer and number 3 is the compiler
            optimizer = rest[1]
            compiler=rest[3]

            dictionary[number]=[' '.join(instructions),rest[1],rest[3]]
    return dictionary


In [ ]:
data= pd.DataFrame.from_dict(create_table("train_dataset.jsonl"),orient='index',columns=['instructions','optimizer','compiler'])

In [ ]:
data.to_csv(r'ML_HW1_data.csv')


In [4]:
data= pd.DataFrame.from_csv('ML_HW1_data.csv')

C:\Users\Theo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [5]:
sum(data['optimizer']=='L')

17924

In [6]:
sum(data['optimizer']=='L')/30000

0.5974666666666667

In [5]:
vectorizer = HashingVectorizer(ngram_range=(5,7),analyzer='word') # multivariate


In [6]:
X_all = vectorizer.fit_transform(data.instructions)
y_all = data.optimizer

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
          test_size=0.2, random_state=1453)

### Try a logistic regression model

In [11]:
model1= LogisticRegression().fit(X_train,y_train)

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
y_pred = model1.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1669  713]
 [ 324 3294]]
              precision    recall  f1-score   support

           H       0.84      0.70      0.76      2382
           L       0.82      0.91      0.86      3618

   micro avg       0.83      0.83      0.83      6000
   macro avg       0.83      0.81      0.81      6000
weighted avg       0.83      0.83      0.82      6000

accuracy is:  0.8271666666666667


In [13]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model1, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.81843593 0.82562407 0.82590307 0.81744557 0.82199161]
Weighted f1 score : 0.822 (+/- 0.01)


### Grid Search for logistic regression

In [16]:
grid_values = {'penalty': ['l1', 'l2'],'C':[0.01,.09,0.1,1,5,10,25,50,100]}
grid_acc = GridSearchCV(model1, param_grid = grid_values,scoring = 'f1_weighted',cv=cv,iid=False,n_jobs=4).fit(X_all,y_all)

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [17]:
for i in range(0,len(grid_acc.cv_results_['params'])):
    print("[%2d] params: %s  \tscore: %.3f +/- %.3f" %(i,
        grid_acc.cv_results_['params'][i],
        grid_acc.cv_results_['mean_test_score'][i],
        grid_acc.cv_results_['std_test_score'][i] ))

a = np.argmax(grid_acc.cv_results_['mean_test_score'])
bestparams = grid_acc.cv_results_['params'][a]
bestscore = grid_acc.cv_results_['mean_test_score'][a]

print("Best configuration [%d] %r  %.3f" %(a,bestparams,bestscore))

[ 0] params: {'C': 0.01, 'penalty': 'l1'}  	score: 0.493 +/- 0.002
[ 1] params: {'C': 0.01, 'penalty': 'l2'}  	score: 0.602 +/- 0.001
[ 2] params: {'C': 0.09, 'penalty': 'l1'}  	score: 0.683 +/- 0.004
[ 3] params: {'C': 0.09, 'penalty': 'l2'}  	score: 0.731 +/- 0.003
[ 4] params: {'C': 0.1, 'penalty': 'l1'}  	score: 0.688 +/- 0.004
[ 5] params: {'C': 0.1, 'penalty': 'l2'}  	score: 0.735 +/- 0.002
[ 6] params: {'C': 1, 'penalty': 'l1'}  	score: 0.797 +/- 0.003
[ 7] params: {'C': 1, 'penalty': 'l2'}  	score: 0.822 +/- 0.004
[ 8] params: {'C': 5, 'penalty': 'l1'}  	score: 0.836 +/- 0.003
[ 9] params: {'C': 5, 'penalty': 'l2'}  	score: 0.847 +/- 0.003
[10] params: {'C': 10, 'penalty': 'l1'}  	score: 0.842 +/- 0.004
[11] params: {'C': 10, 'penalty': 'l2'}  	score: 0.851 +/- 0.003
[12] params: {'C': 25, 'penalty': 'l1'}  	score: 0.844 +/- 0.006
[13] params: {'C': 25, 'penalty': 'l2'}  	score: 0.854 +/- 0.004
[14] params: {'C': 50, 'penalty': 'l1'}  	score: 0.843 +/- 0.006
[15] params: {'C': 

In [8]:
model15= LogisticRegression(C=100).fit(X_train,y_train)

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
y_pred = model15.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1893  489]
 [ 369 3249]]
              precision    recall  f1-score   support

           H       0.84      0.79      0.82      2382
           L       0.87      0.90      0.88      3618

   micro avg       0.86      0.86      0.86      6000
   macro avg       0.85      0.85      0.85      6000
weighted avg       0.86      0.86      0.86      6000

accuracy is:  0.857


### try a linear support vector machines model

In [20]:
model2=svm.SVC(gamma='scale',kernel='linear',cache_size=40000).fit(X_train,y_train)

In [21]:
y_pred = model2.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1796  586]
 [ 322 3296]]
              precision    recall  f1-score   support

           H       0.85      0.75      0.80      2382
           L       0.85      0.91      0.88      3618

   micro avg       0.85      0.85      0.85      6000
   macro avg       0.85      0.83      0.84      6000
weighted avg       0.85      0.85      0.85      6000

accuracy is:  0.8486666666666667


### try 2 decision trees, with different loss criteria classifier


In [34]:
model3=DecisionTreeClassifier().fit(X_train, y_train)

In [35]:
y_pred = model3.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1674  708]
 [ 517 3101]]
              precision    recall  f1-score   support

           H       0.76      0.70      0.73      2382
           L       0.81      0.86      0.84      3618

   micro avg       0.80      0.80      0.80      6000
   macro avg       0.79      0.78      0.78      6000
weighted avg       0.79      0.80      0.79      6000

accuracy is:  0.7958333333333333


In [36]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model3, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.79525905 0.79573074 0.78594184 0.78101215 0.78833776]
Weighted f1 score : 0.789 (+/- 0.01)


In [25]:
model4 = DecisionTreeClassifier(criterion='entropy').fit(X_train,y_train)

In [26]:
y_pred = model4.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1665  717]
 [ 553 3065]]
              precision    recall  f1-score   support

           H       0.75      0.70      0.72      2382
           L       0.81      0.85      0.83      3618

   micro avg       0.79      0.79      0.79      6000
   macro avg       0.78      0.77      0.78      6000
weighted avg       0.79      0.79      0.79      6000

accuracy is:  0.7883333333333333


In [27]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model4, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.78845018 0.78975191 0.78833334 0.77928151 0.79065298]
Weighted f1 score : 0.787 (+/- 0.01)


### try a random forest, consisting of 100 decision trees

In [28]:
model5=RandomForestClassifier(n_estimators=100).fit(X_train,y_train)

In [29]:
y_pred = model5.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1699  683]
 [ 350 3268]]
              precision    recall  f1-score   support

           H       0.83      0.71      0.77      2382
           L       0.83      0.90      0.86      3618

   micro avg       0.83      0.83      0.83      6000
   macro avg       0.83      0.81      0.82      6000
weighted avg       0.83      0.83      0.83      6000

accuracy is:  0.8278333333333333


In [30]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model5, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.82408336 0.83428923 0.8379738  0.81980556 0.82318422]
Weighted f1 score : 0.828 (+/- 0.01)


In [ ]:
### if the compilers affect the optimization. the last part of stratified, which is with compiler icc? seems to have many H, so the score is low.

In [ ]:
### the problem is the low recall of the High optimization. with grid search, logistic regression smv too, seem to solve it.

### Try a knn classifier

In [7]:
model6 = KNeighborsClassifier(n_neighbors=10,weights='distance').fit(X_train,y_train)

In [9]:
y_pred = model6.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1372 1010]
 [1054 2564]]
              precision    recall  f1-score   support

           H       0.57      0.58      0.57      2382
           L       0.72      0.71      0.71      3618

   micro avg       0.66      0.66      0.66      6000
   macro avg       0.64      0.64      0.64      6000
weighted avg       0.66      0.66      0.66      6000

accuracy is:  0.656


In [10]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model6, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.66388183 0.65808142 0.66698607 0.6598116  0.66063526]
Weighted f1 score : 0.662 (+/- 0.01)


### try a naive bayes multinomial classifier

In [11]:
vectorizer = HashingVectorizer(ngram_range=(5,7),analyzer='word',non_negative=True) # non negative values are changed, so we can implement bayes
X_all = vectorizer.fit_transform(data.instructions)
y_all = data.optimizer
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
          test_size=0.2, random_state=1453)

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [12]:
model7 = MultinomialNB(fit_prior=True).fit(X_train,y_train)

In [13]:
y_pred = model7.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy is:  '+str((confusion_matrix(y_test, y_pred)[0][0]+confusion_matrix(y_test, y_pred)[1][1])/(confusion_matrix(y_test, y_pred)[0].sum()+confusion_matrix(y_test, y_pred)[1].sum())))

[[1402  980]
 [ 305 3313]]
              precision    recall  f1-score   support

           H       0.82      0.59      0.69      2382
           L       0.77      0.92      0.84      3618

   micro avg       0.79      0.79      0.79      6000
   macro avg       0.80      0.75      0.76      6000
weighted avg       0.79      0.79      0.78      6000

accuracy is:  0.7858333333333334


In [14]:
cv = StratifiedKFold(n_splits=5,random_state=120,shuffle=True)
scores = cross_val_score(model7, X_all, y_all, cv=cv,scoring='f1_weighted')
print(scores)
print("Weighted f1 score : %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.77139745 0.78008472 0.78310348 0.7770984  0.76437526]
Weighted f1 score : 0.775 (+/- 0.01)


### Predict the optimizer for the blind test set

In [10]:
def create_table_2(name_of_file):
    dictionary=dict()
    with open(name_of_file,"r") as data:
        for number,line in enumerate(data):
            instructions=[]
            instr= line.strip('{"instructions": [').split("],")[0].replace('"','')
            for i in range(len(instr.split(', '))):
                try:
                    instructions.append(instr.split(', ')[i].split()[0])

                except:
                    print('error or instance  '+str(i)+ 'and number  '+str(number)+ 'with text:   '+ str(instr.split(', ')[i].split()))

           

            dictionary[number]=[' '.join(instructions).strip(']}')]
    return dictionary


In [11]:
data_test= pd.DataFrame.from_dict(create_table_2("test_dataset_blind.jsonl"),orient='index',columns=['instructions'])

In [12]:
data_test.head()

,instructions
0,push push push push push push sub mov mov mov ...
1,mov mov mov xor call nop mov call nop test jne...
2,mov neg shr mov ret
3,push push mov push push mov sub mov call test ...
4,est jne mov lea mov mov mov mov call mov mov m...


In [13]:
X_all_test = vectorizer.fit_transform(data_test.instructions)


In [14]:
optimizer_predictions = model15.predict(X_all_test)


In [15]:
optimizer_predictions

array(['L', 'H', 'L', ..., 'L', 'H', 'L'], dtype=object)

In [16]:
sum(optimizer_predictions=='H')

1196

In [17]:
sum(optimizer_predictions=='L')

1804

In [18]:
np.savetxt("ML_HW1_optimizers_predictions.csv", optimizer_predictions,fmt='%s')